## Schema Evolution: Forward Compatibility

Forward compatibility ensures that consumers using the new schema can read data produced with the old schema. This is useful when you want to add new fields to your schema without breaking existing consumers.

### Example: Adding a New Field to the User Schema

Let's evolve the existing User Avro schema by adding a new field `age`. When adding a new field to a schema for forward compatibility, it is important to provide a default value. This ensures that the new schema can read data produced with the old schema, which does not contain the new field.

In [ ]:
import requests
import json

# Schema Registry URL
schema_registry_url = 'http://schema-registry:8081'

# Evolved User Avro Schema
evolved_user_avro_schema = {
    "type": "record",
    "name": "User",
    "fields": [
        {"name": "id", "type": "int"},
        {"name": "name", "type": "string"},
        {"name": "email", "type": "string"},
        {"name": "age", "type": "int", "default": 0}  # New field with default value
    ]
}

# Register Evolved Avro Schema
response = requests.post(
    f"{schema_registry_url}/subjects/user-avro-value/versions",
    headers={"Content-Type": "application/vnd.schemaregistry.v1+json"},
    data=json.dumps({"schema": json.dumps(evolved_user_avro_schema)})
)

if response.status_code == 200:
    print("Evolved Avro schema registered successfully!")
else:
    print(f"Failed to register evolved Avro schema: {response.text}")

In [ ]:
from confluent_kafka import Producer
from confluent_kafka.serialization import StringSerializer, SerializationContext, MessageField
from confluent_kafka.schema_registry import SchemaRegistryClient, Schema
from confluent_kafka.schema_registry.avro import AvroSerializer
import time
import avro.schema

# Kafka Configuration
conf = {
    'bootstrap.servers': "kafka-broker-1:29094,kafka-broker-2:29094"
}

# Schema Registry Configuration
schema_registry_conf = {'url': 'http://schema-registry:8081'}
schema_registry_client = SchemaRegistryClient(schema_registry_conf)

# Evolved User Avro Schema
evolved_user_avro_schema_str = """
{
    "type": "record",
    "name": "User",
    "fields": [
        {"name": "id", "type": "int"},
        {"name": "name", "type": "string"},
        {"name": "email", "type": "string"},
        {"name": "age", "type": "int", "default": 0}
    ]
}
"""

# Create Schema object
evolved_user_schema = Schema(evolved_user_avro_schema_str, "AVRO")
# Create Avro Serializer
avro_serializer = AvroSerializer(schema_registry_client, evolved_user_schema)
# Create Producer Instance
producer = Producer(conf)
# Kafka Topic
topic = "user-avro"

# Produce User Messages with Evolved Schema
for i in range(10):
    user = {'id': i, 'name': f"User {i}", 'email': f"user{i}@example.com", 'age': 25 + i}
    producer.produce(
        topic=topic,
        key=StringSerializer('utf_8')(str(i), SerializationContext(topic, MessageField.KEY)),
        value=avro_serializer(user, SerializationContext(topic, MessageField.VALUE))
    )
    print(f"Produced: {user}")
    producer.flush()  # Ensure delivery
    time.sleep(1)  # Simulate delay between messages

print("All user messages with evolved schema produced successfully!")

In [ ]:
from confluent_kafka import Consumer
from confluent_kafka.serialization import StringDeserializer, SerializationContext, MessageField
from confluent_kafka.schema_registry.avro import AvroDeserializer

# Consumer Configuration
consumer_conf = {
    'bootstrap.servers': "kafka-broker-1:29094,kafka-broker-2:29094",
    'group.id': 'user-avro-consumer-group',
    'auto.offset.reset': 'earliest'
}

# Create Consumer Instance
consumer = Consumer(consumer_conf)
# Subscribe to the topic
consumer.subscribe(["user-avro"])

# Old User Avro Schema (without 'age' field)
old_user_avro_schema_str = """
{
    "type": "record",
    "name": "User",
    "fields": [
        {"name": "id", "type": "int"},
        {"name": "name", "type": "string"},
        {"name": "email", "type": "string"}
    ]
}
"""

# Create Avro Deserializer
old_user_schema = Schema(old_user_avro_schema_str, "AVRO")
avro_deserializer = AvroDeserializer(schema_registry_client, old_user_schema)

# Consume Messages
try:
    while True:
        msg = consumer.poll(1.0)
        if msg is None:
            continue
        if msg.error():
            print(f"Consumer error: {msg.error()}")
            continue
        user = avro_deserializer(msg.value(), SerializationContext(msg.topic(), MessageField.VALUE))
        print(f"Consumed: {user}")
except KeyboardInterrupt:
    pass
finally:
    consumer.close()

print("All user messages with old schema consumed successfully!")